In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
from gurobipy import *

In [ ]:
# load profile
pd.options.plotting.backend = "plotly"
template = "plotly_white"
path="../data/SBAP_FCR_profile_test2.csv"
#profile = pd.read_csv("../data/SBAP_FCR_profile_test.csv", index_col=0, parse_dates=True)
profile = pd.read_csv("../data/SBAP_FCR_profile_test2.csv", index_col=0, parse_dates=True)
#profile = pd.read_csv("../data/ecostor_fcr_wholesale_15min_3.csv", index_col=0, parse_dates=True)
#profile = pd.read_csv("../data/industry_profile_test.csv", index_col=0, parse_dates=True)
#profile = pd.read_csv("../data/myProfile3.csv", index_col=0, parse_dates=True)
profile.plot(template=template, labels={"value": "Power [W]"})

In [ ]:
data1 = pd.read_csv(path, index_col=0, parse_dates=True)

# Extract the data from the columns you want to plot
#x = data1['time']
y1 = data1['power']




plt.subplots(figsize=(15, 5))

# Plot data from Column1
plt.plot(y1*0.001, label='Grid FCR Power', color="blue", linewidth=1)


# Customize labels and title
plt.xlabel('Time (days)', fontsize=34)
plt.ylabel('Power (kW)', fontsize=34)
#plt.title('String-1 Module-4 Temperature Evolution')

# Add grid lines
plt.grid(True, linestyle='--')

# Add a legend
plt.legend(fontsize=26)
plt.xticks(fontsize=26)
plt.yticks(fontsize=26)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d.%m'))

#plt.xlim([data1.index.min(), data1.index.max()])
#plt.ylim([y1.min(), y1.max()])


# Adjust layout
plt.tight_layout()
plt.savefig("SBAP_FCR_Power.png", dpi=300, bbox_inches='tight')
plt.savefig("SBAP_FCR_Power.svg", dpi=300, bbox_inches='tight')
# Show the plot
plt.show()



In [ ]:
# Storage parameters
storage_params_1 = {
    "capacity": 1500,  # kWh
    #"power": 150.0,      # kW
    "power": 600,
    
    "soc_bounds": (0.1, 0.9),
    "soc_start": 0.6,

    "effc": 0.9,     # charge efficiency
    "effd": 0.9,     # discharge efficiency
}
storage_params_2 = {
    "capacity": 1500,  # kWh
    #"power": 150.0,      # kW
    "power": 600,

    "soc_bounds": (0.1, 0.9),
    "soc_start": 0.3,

    "effc": 0.9,     # charge efficiency
    "effd": 0.9,     # discharge efficiency
}

# Initialization of Power profile
initial_inputs = {
    "timestep": 0.25,   #15min timesteps
    "p_ch1": 0,
    "p_dch1": 0,
    "p_ch2": 0,
    "p_dch2": 0,
    "kw": 0.001   #W to kW conversion, Make it equal to "1.0" if the input profile is in kW
    
}

In [ ]:
 ## =====  Parameters  ===== ##
n = len(profile)   # number of timesteps
time = range(0, n-1) # T
dt = initial_inputs["timestep"]               # 15 min timesteps
kw= initial_inputs["kw"]

## Battery1
capacity1  = storage_params_1["capacity"]
max_power1 = storage_params_1["power"]
p_ch1=initial_inputs["p_ch1"]
p_dch1=initial_inputs["p_dch1"]

soc1_min, soc1_max = storage_params_1["soc_bounds"]
soe1_range= soc1_max-soc1_min


effc1  = storage_params_1["effc"]
effd1  = storage_params_1["effd"]

 ## Battery2

capacity2  = storage_params_2["capacity"]
max_power2 = storage_params_2["power"]

p_ch2=initial_inputs["p_ch2"]
p_dch2=initial_inputs["p_dch2"]

soc2_min, soc2_max = storage_params_2["soc_bounds"]
soc2_range= soc2_max-soc2_min 



effc2  = storage_params_2["effc"]
effd2 = storage_params_2["effd"]



   ## =====  Variables  ===== ##
# BESS1: power charge/discharge, energy content
soe1             = [0.0] * len(time)
soc1             = [0.0] * len(time)
power_tu1        = [0.0] * len(time)
power_charge1    = [0.0] * len(time)
power_discharge1 = [0.0] * len(time)
energy_bess1     = [0.0] * len(time)
delta_SOE        = [0.0] * len(time)
power_loss1      = [0.0] * len(time)

 # BESS2: power charge/discharge, energy content
soe2             = [0.0] * len(time)
soc2             = [0.0] * len(time)
power_tu2        = [0.0] * len(time)
power_charge2    = [0.0] * len(time)
power_discharge2 = [0.0] * len(time)
energy_bess2     = [0.0] * len(time)
abs_delta        = [0.0] * len(time)   
power_loss2      = [0.0] * len(time)

AI = [1.0] * len(time)               #Availability index

soc1[0] = storage_params_1["soc_start"]
soc2[0] = storage_params_2["soc_start"]

zero = [0.0] * len(time)
frac = [0.01] * len(time)

for t in range(1,len(time)):
   #print ("Time: ", t)
   if 0.1<soc1[t-1]<0.9:
      power_tu1[t-1]=profile.power[t-1]*kw
      power_tu2[t-1]=0
      power_loss1[t-1]= (0.021*abs(power_tu1[t-1])+(0.005*max_power1))
      power_loss2[t-1] = (0.021*abs(power_tu2[t-1])+(0.005*max_power2))
      soc1[t]= soc1[t-1] + (((power_tu1[t-1])*dt)/(capacity1)) - ((power_loss1[t-1]*dt)/(capacity1))
      soc2[t]= soc2[t-1]
      if soc1[t]>=0.9 or soc1[t]<=0.1:
         power_tu1[t-1]= 0
         power_loss1[t-1] = (0.021*abs(power_tu1[t-1])+(0.005*max_power1))
         soc1[t]=soc1[t-1]
         power_tu2[t-1]=profile.power[t-1]*kw
         power_loss2[t-1]= (0.021*(abs(power_tu2[t-1]))+(0.005*max_power2))
         soc2[t]= soc2[t-1] + (((power_tu2[t-1])*dt)/(capacity2)) - ((power_loss2[t-1]*dt)/(capacity2))
         if soc2[t]>=0.9 or soc2[t]<=0.1:
               soc2[t]=soc2[t-1]
               power_tu2[t-1]= 0
               print("1. Batteries breached limit")
               AI[t-1]= 0
               power_loss1[t-1] = (0.021*abs(power_tu1[t-1])+(0.005*max_power2))
               power_loss2[t-1] = (0.021*abs(power_tu2[t-1])+(0.005*max_power2))
         #else:
         #   soc1[t]=soc1[t-1]
         #   power_tu1[t-1]= 0
         #   power_loss1[t-1] = (0.021*abs(power_tu1[t-1])+(0.005*max_power1))
         #   power_tu2[t-1]=profile.power[t-1]*kw
         #   power_loss2[t-1]= (0.021*(abs(power_tu2[t-1]))+(0.005*max_power2))
         #   soc2[t]= soc2[t-1] + (((power_tu2[t-1])*dt)/(capacity2)) - ((power_loss2[t-1]*dt)/(capacity2))



      #else:
      #   power_tu1[t-1]=profile.power[t-1]*kw
      #   power_loss1[t-1]= (0.021*abs(power_tu1[t-1])+(0.005*max_power1))
      #   soc1[t]= soc1[t-1] + (((power_tu1[t-1])*dt)/(capacity1)) - ((power_loss1[t-1]*dt)/(capacity1))
      #   soc2[t]= soc2[t-1]
      #   power_tu2[t-1]=0
      #   power_loss2[t-1] = (0.021*(power_tu2[t-1])+(0.0075*max_power2))

   elif 0.1<soc2[t-1]<0.9 and not (0.1 < soc1[t-1] < 0.9):
      power_tu1[t-1]= 0
      power_loss1[t-1]= (0.021*abs(power_tu1[t-1])+(0.005*max_power1))
      soc1[t]=soc1[t-1]
      power_tu2[t-1]=profile.power[t-1]*kw
      power_loss2[t-1]= (0.021*abs(power_tu2[t-1])+(0.005*max_power2))
      soc2[t]= soc2[t-1] + (((power_tu2[t-1])*dt)/(capacity2)) - ((power_loss2[t-1]*dt)/(capacity2))
      if soc2[t]>0.9 or soc2[t]<0.1:
         soc2[t]=soc2[t-1]
         power_tu2[t-1]= 0
         print("2. Batteries breached limit")
         AI[t-1]= 0
         power_loss1[t-1]= (0.021*abs(power_tu1[t-1])+(0.005*max_power1))         
         power_loss2[t-1]= (0.021*abs(power_tu2[t-1])+(0.005*max_power2))
         
   else:
      power_tu1[t-1]=0
      power_tu2[t-1]=0
      print("3. Batteries breached limit")
      AI[t-1]= 0
      power_loss1[t-1]= (0.021*abs(power_tu1[t-1])+(0.005*max_power1))
      power_loss2[t-1]= (0.021*abs(power_tu2[t-1])+(0.005*max_power2))
      soc1[t]=soc1[t-1] 
      soc2[t]=soc2[t-1]

   #print("SOC1: ", soc1[t])
   #print("SOC2: ", soc2[t])


In [ ]:
SoE1 = [soc1[t] for t in time]
SoE2 = [soc2[t] for t in time]

fig = go.Figure()
fig.update_layout(template=template)
fig.add_trace(go.Scatter(x=profile.index, y=SoE1, name="SoC 1", line=dict(color="orange")))
fig.add_trace(go.Scatter(x=profile.index, y=SoE2, name="SoC 2", line=dict(color="green")))

fig.update_yaxes(title="State of Charge")
#fig.update_traces(line_shape="hv")
fig.update_layout(
    width=900,  # Specify the width in pixels
    height=500  # Specify the height in pixels
)

In [ ]:

# Data
SoE1 = [soc1[t] for t in time]
SoE2 = [soc2[t] for t in time]

del_SOE=[abs(soc2[t]-soc1[t]) for t in time]

# Create a new figure
fig, ax = plt.subplots(figsize=(15, 8))

# Plot the data
ax.plot(profile.index[:len(SoE1)], SoE1, label="String_1_SOC", color="orange", linewidth=2)
ax.plot(profile.index[:len(SoE2)], SoE2, label="String_2_SOC", color="green", linewidth=2)

#ax2 = ax.twinx()
#ax2.plot(profile.index[:len(SoE1)], del_SOE, label="del_SOC", color="brown", linewidth=1)

# Set labels and title
ax.set_xlabel("Time (days)",fontsize=34)
ax.set_ylabel("State of Charge",fontsize=34)
#ax.set_title("State of Charge Over Time")

ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m'))

# Add grid lines
ax.grid(True, linestyle='--')

# Set labels and title for the second y-axis
#ax2.set_ylabel("delta_SOC", fontsize=24)
ax.legend(fontsize=26, loc="lower right")
ax.tick_params(axis='y', labelsize=26)
plt.yticks(np.arange(0.1, 1.0, 0.1))
ax.tick_params(axis='x', labelsize=26)
#ax2.tick_params(axis='y', labelsize=16)
#plt.xlim([data1.index.min(), data1.index.max()])
#plt.ylim(0.1, 0.9)


# Adjust layout
plt.tight_layout()

# Save the plot as a high-quality image (e.g., PNG or PDF)
plt.savefig("SoE_Plot_consq.png", dpi=300, bbox_inches="tight")
# Or, if you prefer to display the plot interactively, use plt.show()
# plt.show()


In [ ]:
power_tu1 = [power_tu1[t] for t in time]
power_tu2 = [power_tu2[t] for t in time]

fig = go.Figure()
fig.update_layout(template=template)
fig.add_trace(go.Scatter(x=profile.index, y=power_tu1, name="Power 1", line=dict(color="orange")))
fig.add_trace(go.Scatter(x=profile.index, y=power_tu2, name="Power 2", line=dict(color="green")))

fig.update_yaxes(title="String Power (kW)")
#fig.update_traces(line_shape="hv")
fig.update_layout(
    width=900,  # Specify the width in pixels
    height=500  # Specify the height in pixels
)

In [ ]:
# Data
power_tu1 = [power_tu1[t] for t in time]
power_tu2 = [power_tu2[t] for t in time]

# Create a new figure
fig, ax = plt.subplots(figsize=(15, 5))

# Plot the data
ax.plot(profile.index[:len(power_tu1)], power_tu1, label="String_1_Power", color="orange", linewidth=2)
ax.plot(profile.index[:len(power_tu2)], power_tu2, label="String_2_Power", color="green", linewidth=2)
# Set labels and title
ax.set_xlabel("Time (days)",fontsize=36)
ax.set_ylabel("Power (kW)",fontsize=36)
#ax.set_title("State of Charge Over Time")

# Add grid lines
ax.grid(True, linestyle='--')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m'))

# Add a legend
ax.legend(fontsize=30)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)

#plt.xlim([data1.index.min(), data1.index.max()])

# Adjust layout
plt.tight_layout()

# Save the plot as a high-quality image (e.g., PNG or PDF)
plt.savefig("Power_split_consq.png", dpi=300, bbox_inches="tight")
# Or, if you prefer to display the plot interactively, use plt.show()
# plt.show()


In [ ]:
# Create a DataFrame from the lists
data = {'soc1': soc1, 'soc2': soc2, 'power_tu1': power_tu1, 'power_tu2': power_tu2, 'power_loss1': power_loss1, 'power_loss2': power_loss2 }
df = pd.DataFrame(data)

# Specify the path to your new CSV file
output_csv_file = 'output_ConsqDist.csv'

# Save the DataFrame to a CSV file
df.to_csv(output_csv_file, index=False)

In [ ]:
total= sum(AI)
availability_index = total/len(AI)
print("AI: ",availability_index)



In [ ]:
ave_soc1= sum(soc1)/len(soc1)
ave_soc2= sum(soc2)/len(soc2)
print("Average soc1:", ave_soc1)
print("Average soc2:", ave_soc2)
print("System Avergae SOC: ",(ave_soc1+ave_soc2)/2)

In [ ]:
output = pd.read_csv("output_ConsqDist.csv")
profile = profile.drop(profile.index[-1])

total_input= sum(abs(profile.power))
print("total_input =", total_input*0.001)
total_system =sum(abs(output.power_tu1)) + sum(abs(output.power_tu2))
total_loss =sum(abs(output.power_loss1)) + sum(abs(output.power_loss2))
print("total_system =", total_system)

throughput= total_system/(total_input*0.001)
print("throughput =", throughput)
print("total_loss =", total_loss)
print("conversion =", 1-(total_loss/total_system))



In [ ]:
net_input= sum((profile.power))
print(net_input*kw)